Imports

In [ ]:
# from win32com.client import Dispatch
from astropy import coordinates as coord, time
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import Button
from IPython.display import display, clear_output

# Constants
meter = 1
millimeter = 1e-3 * meter
micron = 1e-6 * meter
deg = np.pi / 180
arcsec = deg / 3600
hourangle = deg * 15
secangle = hourangle / 3600

# Telescope parameters
plate_scale = 1 / (2939 * millimeter)
pixel = 9 * micron

Connect to mount

In [ ]:
telescope = Dispatch("ASCOM.PWI4.Telescope")
telescope.Connected = True

print('Coordinate system: %s' % telescope.EquatorialSystem)

Slew to source and iterate offsets to center it

In [ ]:
object_name = 'Arcturus'
ra_offset = 1458 * pixel * plate_scale
dec_offset = -805 * pixel * plate_scale

obj = coord.SkyCoord.from_name(object_name)
obj_now = obj.transform_to(coord.TETE(obstime=time.Time.now(), location=coord.EarthLocation.from_geodetic(lat=41.6620165, lon=-91.5325033)))
# telescope.SlewToCoordinates(obj_now.ra.hour + ra_offset/hourangle, obj_now.dec.deg + dec_offset/deg)
# telescope.Tracking = True

print('Object ICRS coordinates: %s' % obj.to_string('hmsdms'))
print('Slew coordinates: %s' % obj_now.to_string('hmsdms'))

Compute the initial search grid from the geometry of the fiber location

In [ ]:
D = 28.047 * millimeter # 27.2 from my drawing
theta = (26.889+90) * deg

# Guess nominal ra and dec from above geometry
nominal_dec = -D * np.sin(theta) * plate_scale
nominal_ra = D * np.cos(theta) * plate_scale / np.cos(obj.dec.rad)
print('Nominal deltaRA = %.2f arcsec' % (nominal_ra / arcsec))
print('Nominal deltaDec = %.2f arcsec' % (nominal_dec / arcsec))

# Generate grid of deltaRA and deltaDec
delta_ras = np.linspace(-300, 300, 7, endpoint=True) * arcsec
delta_decs = np.linspace(-300, 300, 7, endpoint=True) * arcsec
x, y = np.meshgrid(delta_ras, delta_decs)
xrot = np.zeros(x.shape); yrot = np.zeros(y.shape)
for i in range(len(x)):
        for j in range(len(y)):
            xrot[i,j], yrot[i,j] = np.dot([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]], np.array([x[i,j], y[i,j]]))
delta_ras = xrot.flatten(); delta_decs = yrot.flatten()

# Plot grid
fig,ax = plt.subplots(figsize=(6,6))
ax.grid(); ax.set_axisbelow(True)
ax.scatter((delta_ras+nominal_ra)/arcsec, (delta_decs+nominal_dec)/arcsec, c=range(len(delta_ras)), cmap='jet_r')
for i in range(len(delta_ras)):
    ax.annotate(str(i+1), ((delta_ras[i]+nominal_ra)/arcsec, (delta_decs[i]+nominal_dec)/arcsec), textcoords='offset points',
    xytext=(0, 6))
ax.set_xlabel('deltaRA (arcsec)')
ax.set_ylabel('deltaDec (arcsec)')
for i in range(len(delta_ras)):
    circle = plt.Circle(((nominal_ra+delta_ras[i])/arcsec, (nominal_dec+delta_decs[i])/arcsec), 
        70, color='r', fill=False); ax.add_artist(circle)
    circle = plt.Circle(((nominal_ra+delta_ras[i])/arcsec, (nominal_dec+delta_decs[i])/arcsec),
        70/4, color='b', fill=False); ax.add_artist(circle)
plt.show()

Attempt an initial slew to the nominal guess

In [ ]:
telescope.SlewToCoordinates(obj_now.ra.hour + ra_offset/hourangle + nominal_ra/hourangle, 
    obj_now.dec.deg + dec_offset/deg + nominal_dec/deg)

Grid search

In [ ]:
print('Object Coordinates: %s\n\n' % obj_now.to_string('hmsdms'))

for i in range(len(delta_coords)):
    print('Coordinate Pair %i/%i' % ((i+1), len(delta_coords)))
    print('deltaRA     =     %.2f arcsec,  deltaDec = %.2f arcsec' % ((delta_coords[i][0] + nominal_ra) / arcsec, (delta_coords[i][1] + nominal_dec) / arcsec))
    print('Slew Coordinates: %s\n' % coord.SkyCoord(obj.ra.hour + delta_ras[i]/hourangle, obj.dec.deg + delta_decs[i]/deg, unit=('hour', 'deg')).to_string('hmsdms'))
    telescope.SlewToCoordinates(obj_now.ra.hour + ra_offset/hourangle + nominal_ra/hourangle + delta_coords[i][0]/hourangle, obj_now.dec.deg + dec_offset/deg + nominal_dec/deg + delta_coords[i][1]/deg)
    input('Press Enter to continue...\n')

Coordinate Pair 27/49
deltaRA     =     -1033.02 arcsec,  deltaDec = -1933.96 arcsec
Slew Coordinates: 14h15m33.64189096s +19d07m58.29610832s

Slew to best position

In [ ]:
best_position = 27
print('Best position: %i' % best_position)
print('deltaRA = %.2f arcsec, deltaDec = %.2f arcsec' % ((delta_ras[best_position-1] + nominal_ra) / arcsec, (delta_decs[best_position-1] + nominal_dec) / arcsec))

telescope.SlewToCoordinates(obj_now.ra.hour + ra_offset/hourangle + delta_ras[best_position-1]/hourangle, obj_now.dec.deg + dec_offset/deg + delta_decs[best_position-1]/deg)

Generate new grid around best position

In [ ]:
# Set nominal position to best fit from above grid
nominal_ra = delta_ras[best_position-1] + nominal_ra
nominal_dec = delta_decs[best_position-1] + nominal_dec
print('Nominal deltaRA = %.2f arcsec' % (nominal_ra / arcsec))
print('Nominal deltaDec = %.2f arcsec' % (nominal_dec / arcsec))

# Generate grid of deltaRA and deltaDec
delta_ras = np.linspace(-60, 60, 7, endpoint=True) * arcsec
delta_decs = np.linspace(-60, 60, 7, endpoint=True) * arcsec
x, y = np.meshgrid(delta_ras, delta_decs)
xrot = np.zeros(x.shape); yrot = np.zeros(y.shape)
for i in range(len(x)):
        for j in range(len(y)):
            xrot[i,j], yrot[i,j] = np.dot([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]], np.array([x[i,j], y[i,j]]))
delta_ras = xrot.flatten(); delta_decs = yrot.flatten()

# Plot grid
fig,ax = plt.subplots(figsize=(6,6))
ax.grid(); ax.set_axisbelow(True)
ax.scatter((delta_ras+nominal_ra)/arcsec, (delta_decs+nominal_dec)/arcsec, c=range(len(delta_ras)), cmap='jet_r')
for i in range(len(delta_ras)):
    ax.annotate(str(i+1), ((delta_ras[i]+nominal_ra)/arcsec, (delta_decs[i]+nominal_dec)/arcsec), textcoords='offset points',
    xytext=(0, 6))
ax.set_xlabel('deltaRA (arcsec)')
ax.set_ylabel('deltaDec (arcsec)')
for i in range(len(delta_ras)):
    '''circle = plt.Circle(((nominal_ra+delta_ras[i])/arcsec, (nominal_dec+delta_decs[i])/arcsec), 
        70, color='r', fill=False); ax.add_artist(circle)'''
    circle = plt.Circle(((nominal_ra+delta_ras[i])/arcsec, (nominal_dec+delta_decs[i])/arcsec),
        70/4, color='b', fill=False); ax.add_artist(circle)
plt.show()

Attempt an initial slew to the nominal guess

In [ ]:
telescope.SlewToCoordinates(obj_now.ra.hour + ra_offset/hourangle + nominal_ra/hourangle, 
    obj_now.dec.deg + dec_offset/deg + nominal_dec/deg)

Grid search

In [ ]:
print('Object Coordinates: %s\n\n' % obj_now.to_string('hmsdms'))

for i in range(len(delta_coords)):
    print('Coordinate Pair %i/%i' % ((i+1), len(delta_coords)))
    print('deltaRA     =     %.2f arcsec,  deltaDec = %.2f arcsec' % ((delta_coords[i][0] + nominal_ra) / arcsec, (delta_coords[i][1] + nominal_dec) / arcsec))
    print('Slew Coordinates: %s\n' % coord.SkyCoord(obj.ra.hour + delta_ras[i]/hourangle, obj.dec.deg + delta_decs[i]/deg, unit=('hour', 'deg')).to_string('hmsdms'))
    telescope.SlewToCoordinates(obj_now.ra.hour + ra_offset/hourangle + nominal_ra/hourangle + delta_coords[i][0]/hourangle, obj_now.dec.deg + dec_offset/deg + nominal_dec/deg + delta_coords[i][1]/deg)
    input('Press Enter to continue...\n')

Slew to best position

In [ ]:
best_position = 23
print('Best position: %i' % best_position)
print('deltaRA = %.2f arcsec, deltaDec = %.2f arcsec' % ((delta_ras[best_position-1] + nominal_ra) / arcsec, (delta_decs[best_position-1] + nominal_dec) / arcsec))

telescope.SlewToCoordinates(obj_now.ra.hour + ra_offset/hourangle + delta_ras[best_position-1]/hourangle, obj_now.dec.deg + dec_offset/deg + delta_decs[best_position-1]/deg)

In [ ]:
# Make a button to get next coordinate pair and slew telescope to those coordinates
coord_index = 0

button = Button(description='Next Coordinate Pair')
display(button)
print('Object Coordinates: %s\n\n' % obj.to_string('hmsdms'))

def on_button_clicked(b):
    global coord_index

    clear_output()
    display(button)
    print('Object Coordinates: %s\n\n' % obj.to_string('hmsdms'))
    print('Coordinate Pair %i/%i' % ((coord_index+1), len(delta_coords)))
    print('deltaRA     =     %.2f sec,  deltaDec = %.2f arcsec' % (delta_coords[coord_index][0] / secangle, delta_coords[coord_index][1] / arcsec))
    print('Slew Coordinates: %s\n' % coord.SkyCoord(obj.ra.hour + delta_coords[coord_index][0]/hourangle, obj.dec.deg + delta_coords[coord_index][1]/deg, unit=('hour', 'deg')).to_string('hmsdms'))
    telescope.SlewToCoordinates(obj_now.ra.hour + ra_offset/hourangle + delta_coords[coord_index][0]/hourangle, obj_now.dec.deg + dec_offset/deg + delta_coords[coord_index][1]/deg)
    coord_index += 1

button.on_click(on_button_clicked)

